In [1]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector

In [2]:
def construct_qcc_circuit(entanglers: list):
    '''This function defines the QCC ansatz circuit for VQE. Here we construct exponential blocks using
    entanglers from QMF state as a proof of principle demonstration.
    
    Args:
        entanglers: list storing Pauli words for construction of qcc_circuit.
        backend: statevector, qasm simulator or a real backend.
        truncation: a threshold number to truncate the blocks. Default: None.
    Returns:
        qcc_circuit
    '''
    num_blocks = len(entanglers)
    p = ParameterVector('p', num_blocks)
    
    num_qubits = len(entanglers[0])
    qcc_circuit = QuantumCircuit(num_qubits)
    for i in range(num_blocks):
        circuit = QuantumCircuit(num_qubits)
        key = entanglers[i]
        coupler_map = []
        
        # We first construct coupler_map according to the key.
        for j in range(num_qubits):
            if key[num_qubits-1-j] != 'I':
                coupler_map.append(j)
                
        # Then we construct the circuit.
        if len(coupler_map) == 1:
            # there is no CNOT gate.
            c = coupler_map[0]
            if key[num_qubits-1-c] == 'X':
                circuit.h(c)
                circuit.rz(p[i], c)
                circuit.h(c)
            elif key[num_qubits-1-c] == 'Y':
                circuit.rx(-np.pi/2, c)
                circuit.rz(p[i], c)
                circuit.rx(np.pi/2, c)
                
            qcc_circuit += circuit
        else:
            # Here we would need CNOT gate.
            for j in coupler_map:
                if key[num_qubits-1-j] == 'X':
                    circuit.h(j)
                elif key[num_qubits-1-j] == 'Y':
                    circuit.rx(-np.pi/2, j)
                    
            for j in range(len(coupler_map) - 1):
                circuit.cx(coupler_map[j], coupler_map[j+1])
                
            param_gate = QuantumCircuit(num_qubits)
            param_gate.rz(p[i], coupler_map[-1])
            
            #qcc_circuit += circuit + param_gate + circuit.inverse()
            qcc_circuit.compose(circuit, inplace=True)
            qcc_circuit.compose(param_gate, inplace=True)
            qcc_circuit.compose(circuit.inverse(), inplace=True)
    
    return qcc_circuit

In [3]:
hf_circuit = QuantumCircuit(8)

hf_circuit.x(0)
hf_circuit.x(1)
hf_circuit.x(2)
hf_circuit.x(3)

hf_circuit.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     ├───┤
q_3: ┤ X ├
     └───┘
q_4: ─────
          
q_5: ─────
          
q_6: ─────
          
q_7: ─────

In [4]:
entanglers = ['XXIIIIXY', 'IIXXXYII', 'IXXIXIIY', 'XIIXIXYI',
              'XXIIXYII', 'IXIXIXIY', 'XIXIXIYI', 'IIXXIIXY']

parameterized_circuit = hf_circuit.compose(construct_qcc_circuit(entanglers))
parameterized_circuit.draw()

┌───┐┌──────────┐                                                      »
q_0: ┤ X ├┤ Rx(-π/2) ├──■────────────────────────────────────────────────■──»
     ├───┤└──┬───┬───┘┌─┴─┐                                            ┌─┴─┐»
q_1: ┤ X ├───┤ H ├────┤ X ├──■──────────────────────────────────────■──┤ X ├»
     ├───┤┌──┴───┴───┐└───┘  │                                      │  └───┘»
q_2: ┤ X ├┤ Rx(-π/2) ├──■────┼──────────────────────────────────────┼───────»
     ├───┤└──┬───┬───┘┌─┴─┐  │                                      │       »
q_3: ┤ X ├───┤ H ├────┤ X ├──┼────■─────────────────────────────────┼────■──»
     ├───┤   └───┘    └───┘  │  ┌─┴─┐                               │  ┌─┴─┐»
q_4: ┤ H ├───────────────────┼──┤ X ├─────■────────────────────■────┼──┤ X ├»
     ├───┤                   │  └───┘   ┌─┴─┐    ┌──────────┐┌─┴─┐  │  ├───┤»
q_5: ┤ H ├───────────────────┼──────────┤ X ├────┤ Rz(p[1]) ├┤ X ├──┼──┤ H ├»
     ├───┤                 ┌─┴─┐        └───┘    └──────────┘└───┘┌─┴─┐├───┤»
q_6: ┤ H ├─────────────────┤ X ├──■───────────────────■───────────┤ X ├┤ H ├»
     ├───┤                 └───┘┌─┴─┐┌──────────┐   ┌─┴─┐    ┌───┐├───┤└───┘»
q_7: ┤ H ├──────────────────────┤ X ├┤ Rz(p[0]) ├───┤ X ├────┤ H ├┤ H ├─────»
     └───┘                      └───┘└──────────┘   └───┘    └───┘└───┘     »
«     ┌─────────┐┌──────────┐                                               »
«q_0: ┤ Rx(π/2) ├┤ Rx(-π/2) ├───────■───────────────────────────────────────»
«     └──┬───┬──┘├──────────┤       │                                       »
«q_1: ───┤ H ├───┤ Rx(-π/2) ├───────┼────■──────────────────────────────────»
«        └───┘   ├─────────┬┘┌───┐  │  ┌─┴─┐                                »
«q_2: ─────■─────┤ Rx(π/2) ├─┤ H ├──┼──┤ X ├──■─────────────────────────────»
«        ┌─┴─┐   └──┬───┬──┘ ├───┤┌─┴─┐└───┘  │                             »
«q_3: ───┤ X ├──────┤ H ├────┤ H ├┤ X ├──■────┼─────────────────────────────»
«        ├───┤      ├───┤    └───┘└───┘  │  ┌─┴─┐                           »
«q_4: ───┤ H ├──────┤ H ├────────────────┼──┤ X ├──■─────────────────────■──»
«        ├───┤      └───┘              ┌─┴─┐└───┘  │                     │  »
«q_5: ───┤ H ├─────────────────────────┤ X ├──■────┼────────────────■────┼──»
«        ├───┤                         └───┘┌─┴─┐  │  ┌──────────┐┌─┴─┐  │  »
«q_6: ───┤ H ├──────────────────────────────┤ X ├──┼──┤ Rz(p[2]) ├┤ X ├──┼──»
«        └───┘                              └───┘┌─┴─┐├──────────┤└───┘┌─┴─┐»
«q_7: ───────────────────────────────────────────┤ X ├┤ Rz(p[3]) ├─────┤ X ├»
«                                                └───┘└──────────┘     └───┘»
«                    ┌─────────┐┌──────────┐                           »
«q_0: ────────────■──┤ Rx(π/2) ├┤ Rx(-π/2) ├───────────────────────────»
«                 │  └─────────┘├─────────┬┘┌──────────┐               »
«q_1: ────────────┼───────■─────┤ Rx(π/2) ├─┤ Rx(-π/2) ├───────────────»
«                 │     ┌─┴─┐   └──┬───┬──┘ ├──────────┤               »
«q_2: ───────■────┼─────┤ X ├──────┤ H ├────┤ Rx(-π/2) ├──■────────────»
«            │  ┌─┴─┐   ├───┤      ├───┤    └──────────┘┌─┴─┐          »
«q_3: ──■────┼──┤ X ├───┤ H ├──────┤ H ├────────────────┤ X ├──■───────»
«       │  ┌─┴─┐├───┤   ├───┤      └───┘                └───┘  │       »
«q_4: ──┼──┤ X ├┤ H ├───┤ H ├──────────────────────────────────┼───────»
«     ┌─┴─┐├───┤├───┤   └───┘                                  │       »
«q_5: ┤ X ├┤ H ├┤ H ├──────────────────────────────────────────┼───────»
«     ├───┤├───┤└───┘                                        ┌─┴─┐     »
«q_6: ┤ H ├┤ H ├─────────────────────────────────────────────┤ X ├──■──»
«     ├───┤├───┤                                             └───┘┌─┴─┐»
«q_7: ┤ H ├┤ H ├──────────────────────────────────────────────────┤ X ├»
«     └───┘└───┘                                                  └───┘»
«                                                                              »
«q_0: ─────────────────────────

In [ ]:
# Optimized parameter from equilibrium bond distance of H2O:

parameters = [0.0944527, 0.04799566, -0.0590973, -0.05908328, 0.04114604, 0.02695483, 0.02604318, 0.03485649]